In [11]:
from random import randint
from datetime import datetime
from typing import Union, Dict
import requests
from PIL import Image
from io import BytesIO

In [12]:
def api_query(api_link: str,  agent_id: str) -> Union[Dict,None]:
    """Function which executes query via an api link using a provided agent_id as an identifier to avoid rejection of query request

    Args:
        api_link (str): API Link which requests is to be made
        agent_id (str): Id used for request header

    Returns:
        Dictioanry containing request content. None when exception are encountered.
    """
    req_headers = {"User-agent": agent_id}
    try:
        res = requests.get(api_link,
                            headers=req_headers,
                            timeout=5)
        # Raise if HTTPError occured
        res.raise_for_status()

        # Check the status code before extending the number of posts
        if res.status_code == 200:
            print("Request sucessful with %s", res.status_code)
            the_json = res.json()
        return the_json['items']

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)
    return None

def query_traffic_metadata(api_link: str = "https://api.data.gov.sg/v1/transport/traffic-images", agent_id: str= "test_qzq"): 
    cctv_feed = api_query(api_link=api_link, agent_id=agent_id)
    all_traffic_metadata_dict = {}
    if not cctv_feed:
        print("No response received")
        return {}
    number_of_cameras = len(cctv_feed[0]["cameras"])

    for i in range(number_of_cameras):
        camera_feed = cctv_feed[0]["cameras"][i]

        # Process information to dictionary
        metadata_dict = {"timestamp" : camera_feed["timestamp"],
                "image_url": camera_feed["image"],
                "lat": camera_feed["location"]["latitude"],
                "lon": camera_feed["location"]["longitude"],
                "md5": camera_feed["image_metadata"]["md5"]
        }
        all_traffic_metadata_dict[camera_feed["camera_id"]] = metadata_dict
    return all_traffic_metadata_dict


def get_camera_feed(metadata_dict: Dict, camera_id: str="1005", agent_id: str="test_qzq"):
    try:
        # Extract the camera id from metadata_dict to obtain its metadata of interest
        specific_camera_metadata_dict = metadata_dict[camera_id]

    except KeyError:
        print("Invalid id received. No information obtained")

    # Randomised agent id
    rand_agent_id = agent_id + str(randint(0,500))
    try:
        response = requests.get(specific_camera_metadata_dict["image_url"],
                        stream=True,
                        headers={'User-agent': rand_agent_id},
                        timeout=5
                        )

        # Raise if HTTPError occured
        response.raise_for_status()
        # Write to a file if success and log the status
        img = Image.open(BytesIO(response.content))
        
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)
    return None

In [10]:
query_traffic_metadata()

Request sucessful with %s 200


{'1005': {'timestamp': '2023-11-13T22:17:02+08:00',
  'image_url': 'https://images.data.gov.sg/api/traffic-images/2023/11/af402ba5-3087-4d12-af13-c36c5b59ed88.jpg',
  'lat': 1.363519886,
  'lon': 103.905394,
  'height': 240,
  'width': 320,
  'md5': '3af694446eb89c96903264bdab26a2ef'},
 '1006': {'timestamp': '2023-11-13T22:17:02+08:00',
  'image_url': 'https://images.data.gov.sg/api/traffic-images/2023/11/69d20427-cfe7-4aec-a249-44edaceb2d34.jpg',
  'lat': 1.357098686,
  'lon': 103.902042,
  'height': 240,
  'width': 320,
  'md5': '74c5e54ada8a51e6540236d90b58ae68'},
 '1711': {'timestamp': '2023-11-13T22:17:02+08:00',
  'image_url': 'https://images.data.gov.sg/api/traffic-images/2023/11/00dccc61-dd43-4c5f-a1aa-3e9ba424a37a.jpg',
  'lat': 1.35296,
  'lon': 103.85719,
  'height': 1080,
  'width': 1920,
  'md5': '6c2d7bb229f92af0014454697a602714'},
 '2702': {'timestamp': '2023-11-13T22:17:02+08:00',
  'image_url': 'https://images.data.gov.sg/api/traffic-images/2023/11/3170572f-7b2b-4c2b-b